# get_hrdps

This routine grabs the latest High Resolution Deterministic Prediction System (HRDPS) model data from MSC Datamart and outputs png plots into designated product folders for each ACMWF product.

**Datamart:**

https://eccc-msc.github.io/open-data/msc-data/nwp_hrdps/readme_hrdps_en/

https://dd.weather.gc.ca/model_hrdps/

**Outputs:**

Hourly Precipitation -> *BC/AB, South Coast, South Interior, Alberta Rockies, Precipitation Type*

12Hr Precipitation -> *South Coast, South Interior, Alberta Rockies*

Temperatures -> *Surface*


**Maintenance:**

For bug reports, suggestions, inquiries, contact Andrew.Loeppky@gmail.com

In [1]:
from herbie import Herbie
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "heavy"
from toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd

from matplotlib.patches import Rectangle
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pytz
import os, shutil
import json

import warnings

# supress because removal of old datafiles is handled outside herbie
warnings.filterwarnings("ignore")

from bmwflib import (
    clear_directory,
    get_var,
    plot_cities,
    make_figure,
    make_basemap,
    make_title,
)

%load_ext jupyter_black

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ WARNING: Unable to create config file               │
 │      /Users/andrew/.config/herbie/config.toml        │
 │ Herbie will use standard default settings.           │
 │ Consider setting env variable HERBIE_CONFIG_PATH.    │
 ╰──────────────────────────────────────────────────────╯



In [2]:
def plot_temps(fxx, ds, config):
    """
    plots 2m temperatures, saves hourly
    """
    fig, ax = make_figure()
    ax.set_extent([-140, -110, 46, 65])

    # basemap
    make_basemap(ax)

    # configure colormaps
    t_cmap = mpl.colors.ListedColormap(
        [
            "#FFFAFA",
            "#C0C0C0",
            "#4B0082",
            "#8B008B",
            "#9400D3",
            "#9370DB",
            "#0000CD",
            "#1E90FF",
            "#00BFFF",
            "#F0E68C",
            "#FFFF00",
            "#FFA500",
            "#FF4500",
            "#d62728",
            "#FF69B4",
            "#800000",
            "#000000",
        ]
    )
    t_bounds = [
        -40,
        -35,
        -30,
        -25,
        -20,
        -15,
        -10,
        -5,
        -1,
        1,
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
    ]
    t_norm = mpl.colors.BoundaryNorm(t_bounds, t_cmap.N)

    # plot 2m temperature
    t2m = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds.t2m,
        cmap=t_cmap,
        norm=t_norm,
        antialiased=True,
        transform=pc,
        alpha=0.8,
    )

    # 2m temp colorbar
    fig.text(0.141, 0.689, "($^o$C)", color="black", size=10)
    fig.text(0.14, 0.69, "($^o$C)", color="white", size=10)

    t_cbar = fig.colorbar(
        t2m,
        cmap=t_cmap,
        norm=t_norm,
        ax=ax,
        spacing="uniform",
        shrink=0.6,
        location="left",
        aspect=20,
        anchor=(0.15, 0.25),
        pad=-0.18,
    )
    t_cbar.ax.set_yticks(t_bounds)
    t_cbar.ax.tick_params(
        labelsize=10,
        labelcolor="white",
        length=0,
        labelright=True,
        labelleft=False,
    )

    # cities
    plot_cities(ax)

    # title
    make_title(fig, ax, ds, "Temperature\n2 metres", offset=0.02)

    # save and close figure
    fig.savefig(
        os.path.join(
            config["plots"]["surf_temp"],
            f"temp2m_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )
    fig.clf()

    return None

In [3]:
def plot_precip1hr(fxx, ds, config):
    """
    plots 1hr precip, saves hourly to the BC/AB, south coast,
    south interior, alberta rockies folders
    """
    fig, ax = make_figure()

    # basemap
    make_basemap(ax)

    # plot precip rate
    pr_cmap = mpl.colors.ListedColormap(
        [
            "#90EE90",
            "#ADFF2F",
            "#3CB371",
            "#2E8B57",
            "#FFFF00",
            "#FFA500",
            "#B22222",
            "#8B008B",
        ]
    )
    pr_bounds = [0.1, 1, 2, 5, 7.5, 10, 15, 20, 25]
    pr_labels = ["0.1", "1", "2", "5", "7.5", "10", "15", "20", "25+"]
    pr_norm = mpl.colors.BoundaryNorm(pr_bounds, pr_cmap.N)
    pr = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds["APCP-Accum1h"].where(ds["APCP-Accum1h"] >= 0.1),
        cmap=pr_cmap,
        norm=pr_norm,
        antialiased=True,
        transform=pc,
    )

    # precip colorbar
    fig.text(0.151, 0.699, "(mm/h)", color="black", size=10)
    fig.text(0.15, 0.7, "(mm/h)", color="white", size=10)

    pr_cbar = fig.colorbar(
        pr,
        cmap=pr_cmap,
        norm=pr_norm,
        ax=ax,
        spacing="uniform",
        shrink=0.45,
        location="left",
        aspect=10,
        anchor=(0.15, 0.45),
        pad=-0.16,
    )
    pr_cbar.ax.tick_params(
        labelsize=10,
        labelcolor="white",
        length=0,
        labelright=True,
        labelleft=False,
    )
    pr_cbar.ax.set_yticklabels(pr_labels)

    # cities
    plot_cities(ax)

    # title
    make_title(fig, ax, ds, "Hourly Precipitation", offset=0.03)

    ax.set_extent([-140, -110, 46, 65])

    # BC/AB
    fig.savefig(
        os.path.join(
            config["plots"]["hpcp_bcab"],
            f"pcp1hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )

    # South Coast
    ax.set_extent([-129, -120, 47.5, 53])
    fig.savefig(
        os.path.join(
            config["plots"]["hpcp_southcoast"],
            f"pcp1hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )

    # South Interior
    ax.set_extent([-125, -113, 48.5, 56])
    fig.savefig(
        os.path.join(
            config["plots"]["hpcp_southint"],
            f"pcp1hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )

    # Alberta Rockies
    ax.set_extent([-128, -111, 48.5, 59])
    fig.savefig(
        os.path.join(
            config["plots"]["hpcp_rockies"],
            f"pcp1hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )
    fig.clf()

    return None

In [4]:
def plot_precip12hr(fxx, ds, config):
    """
    plots 1hr precip, saves hourly to the BC/AB, south coast,
    south interior, alberta rockies folders
    """
    fig, ax = make_figure()

    # basemap
    make_basemap(ax)

    # plot precip rate
    pr_cmap = mpl.colors.ListedColormap(
        [
            "#90EE90",
            "#ADFF2F",
            "#3CB371",
            "#2E8B57",
            "#FFFF00",
            "#FFA500",
            "#E9967A",
            "#B22222",
            "#8B008B",
        ]
    )
    pr_bounds = [1, 2.5, 5, 15, 25, 50, 75, 100, 150, 200]
    pr_labels = ["1", "2.5", "5", "15", "25", "50", "75", "100", "150", "200+"]
    pr_norm = mpl.colors.BoundaryNorm(pr_bounds, pr_cmap.N)
    pr = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds["APCP-Accum12h"].where(ds["APCP-Accum12h"] >= 1),
        cmap=pr_cmap,
        norm=pr_norm,
        antialiased=True,
        transform=pc,
    )

    # precip colorbar
    fig.text(0.151, 0.699, "(mm)", color="black", size=10)
    fig.text(0.15, 0.7, "(mm)", color="white", size=10)

    pr_cbar = fig.colorbar(
        pr,
        cmap=pr_cmap,
        norm=pr_norm,
        ax=ax,
        spacing="uniform",
        shrink=0.45,
        location="left",
        aspect=10,
        anchor=(0.15, 0.45),
        pad=-0.16,
    )
    pr_cbar.ax.tick_params(
        labelsize=10,
        labelcolor="white",
        length=0,
        labelright=True,
        labelleft=False,
    )
    pr_cbar.ax.set_yticklabels(pr_labels)

    # cities
    plot_cities(ax)

    # title
    make_title(fig, ax, ds, "Total Precip - 12h", offset=0.03)

    # South Coast
    ax.set_extent([-129, -120, 47.5, 53])
    fig.savefig(
        os.path.join(
            config["plots"]["12hpcp_southcoast"],
            f"pcp12hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )

    # South Interior
    ax.set_extent([-125, -113, 48.5, 56])
    fig.savefig(
        os.path.join(
            config["plots"]["12hpcp_southint"],
            f"pcp12hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )

    # Alberta Rockies
    ax.set_extent([-128, -111, 48.5, 59])
    fig.savefig(
        os.path.join(
            config["plots"]["12hpcp_rockies"],
            f"pcp12hr_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )
    fig.clf()

    return None

In [5]:
def plot_precip_type(fxx, ds, config):
    """
    plots 1hr precip type, saves hourly to the /Precipitation Type directory
    """
    fig, ax = make_figure()
    ax.set_extent([-140, -110, 46, 65])

    # basemap
    make_basemap(ax)

    def do_ptype(ax, pr_cmap, ptype, anchor, label):
        """
        subfunction to plot precipitation type on ax

        1=Rain;
        2=Rain/snow;
        3=Freezing rain;
        4=Ice pellets;
        5=Snow;
        6=None;
        7=Drizzle;
        8=Freezing drizzle;
        9=Freezing rain/ice pellets
        """

        pr_bounds = [0.1, 0.5, 1, 3, 6]
        pr_labels = ["0.1", "0.5", "1.0", "3.0", "6.0"]
        pr_norm = mpl.colors.BoundaryNorm(pr_bounds, pr_cmap.N)
        pr = ax.pcolormesh(
            ds.longitude,
            ds.latitude,
            ds["APCP-Accum1h"]
            .where(ds["APCP-Accum1h"] >= 0.1)
            .where(ds["ptype"].astype(int) == ptype),
            cmap=pr_cmap,
            norm=pr_norm,
            antialiased=True,
            transform=pc,
        )

        pr_cbar = fig.colorbar(
            pr,
            cmap=pr_cmap,
            norm=pr_norm,
            ax=ax,
            label=label,
            spacing="uniform",
            shrink=0.075,
            location="left",
            aspect=2,
            anchor=anchor,
            pad=-0.16,
        )
        pr_cbar.ax.tick_params(
            labelsize=10,
            labelcolor="white",
            length=0,
            labelright=True,
            labelleft=False,
        )
        pr_cbar.ax.set_yticklabels(pr_labels)
        pr_cbar.ax.set_ylabel(
            label, rotation=90, labelpad=15, color="white", weight="bold"
        )

        return None

    # plot rain
    rain_cmap = mpl.colors.ListedColormap(
        ["limegreen", "forestgreen", "green", "darkgreen"]
    )
    do_ptype(ax, rain_cmap, 1, (0.0, 0.15), "Rain")

    # plot mixed precip
    mixed_cmap = mpl.colors.ListedColormap(["yellow", "gold", "orange", "darkorange"])
    do_ptype(ax, mixed_cmap, 2, (0.1, 0.3), "Mixed Precip")

    # plot snow
    snow_cmap = mpl.colors.ListedColormap(
        ["lightskyblue", "deepskyblue", "blue", "darkblue"]
    )
    do_ptype(ax, snow_cmap, 5, (0.2, 0.45), "Snow")

    # plot freezing rain
    frain_cmap = mpl.colors.ListedColormap(
        ["lightcoral", "indianred", "red", "darkred"]
    )
    do_ptype(ax, frain_cmap, 3, (0.3, 0.6), "Freezing Rain")

    # plot ice pellets
    ip_cmap = mpl.colors.ListedColormap(
        ["pink", "deeppink", "mediumvioletred", "darkviolet"]
    )
    do_ptype(ax, ip_cmap, 4, (0.4, 0.75), "Ice Pellets")

    # cities
    plot_cities(ax)

    # units
    fig.text(0.121, 0.719, "(mm)", color="black", size=10)
    fig.text(0.12, 0.72, "(mm)", color="white", size=10)

    # title
    make_title(fig, ax, ds, "Precipitation\nType")

    fig.savefig(
        os.path.join(
            config["plots"]["hpcp_ptype"],
            f"ptype_{ds.model}_{pd.Timestamp(ds.time.values).strftime('%Y_%_h_%d')}_{pd.Timestamp(ds.time.values).strftime('%H')}Z_P{fxx}",
        )
    )
    fig.clf()

    return None

In [6]:
def do_hrdps(run, fxx, config):
    """
    creates all hrdps dependent plots for forecast time fxx
    """
    # clear the hrdps data folder
    clear_dir = [clear_directory(file) for file in list(config["data"].values())]

    temp = get_var(run, "hrdps", fxx, "TMP", "AGL-2m")
    pcp1h = get_var(run, "hrdps", fxx, "APCP-Accum1h", "Sfc")
    ptype = get_var(run, "hrdps", fxx, "PTYPE", "Sfc")

    # plot 12hr accumulation after forecast hour 12 in 6 hr increments
    if (fxx >= 12) and (fxx % 6 == 0):
        pcp12h = get_var(run, "hrdps", fxx, "APCP-Accum12h", "Sfc")
        ds = xr.merge([temp, pcp1h, pcp12h, ptype])
        plot_precip12hr(fxx, ds, config)
    else:
        ds = xr.merge([temp, pcp1h, ptype])

    # convert temps from kelvin to degC
    ds["t2m"] -= 273.15

    # generate plots
    plot_temps(fxx, ds, config)
    plot_precip1hr(fxx, ds, config)
    plot_precip_type(fxx, ds, config)

    return None

In [7]:
def main(tstep=range(1, 49, 1)):
    """
    main function to generate all gdps plots for a forecast run
    (default 48h with 1h increments)
    """
    # time of last model run (0000, 0600, 1200, 1800 Z)
    now = pd.Timestamp.utcnow().floor("6h").tz_localize(None)

    # config determines the directories in which to save each graphic
    with open("..//config/hrdps_config.json") as f:
        config = json.load(f)

    clear_dir = [clear_directory(file) for file in list(config["plots"].values())]
    hrdps = [do_hrdps(now, fxx, config) for fxx in tstep]

    return None

In [9]:
if __name__ == "__main__":
    main()

💔 Did not find ┊ model=hrdps ┊ product=continental/2.5km ┊ 2025-Jun-07 18:00 UTC F01


FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrew/data/hrdps/20250607/20250607T18Z_MSC_HRDPS_TMP_AGL-2m_RLatLon0.0225_PT001H.grib2'